In [1]:
import torch
import numpy as np
import re
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# Setup experiment data

In [2]:
# Dataset, Model, Loss, Hierarchy, Results Filenames
exp_info = [
    # Imagenet 100
    ['Imagenet 100', 'Softmax', 'CE', None, ['experiments/imagenet100/softmax/R0/exp.result',
                                             'experiments/imagenet100/softmax/R1/exp.result',
                                             'experiments/imagenet100/softmax/R2/exp.result', ]],
    ['Imagenet 100', 'Cascade', 'Soft1-1', 'NBDT', ['experiments/imagenet100/cascade/nbdt-binary/softpred_R0/exp.result',
                                                    'experiments/imagenet100/cascade/nbdt-binary/softpred_R1/exp.result',
                                                    'experiments/imagenet100/cascade/nbdt-binary/softpred_R2/exp.result',
                                                    ]],
    ['Imagenet 100', 'Cascade', 'Soft1-1', '2 Lvl WN', ['experiments/imagenet100/cascade/two-lvl-wn/softpred_R0/exp.result',
                                                        'experiments/imagenet100/cascade/two-lvl-wn/softpred_R1/exp.result',
                                                        'experiments/imagenet100/cascade/two-lvl-wn/softpred_R2/exp.result', ]],
    ['Imagenet 100', 'Cascade', 'Soft1-1', 'Pruned WN', ['experiments/imagenet100/cascade/pruned-wn/softpred_R0/exp.result',
                                                         'experiments/imagenet100/cascade/pruned-wn/softpred_R1/exp.result',
                                                         'experiments/imagenet100/cascade/pruned-wn/softpred_R2/exp.result', ]],
    ['Imagenet 100', 'Cascade', 'Soft1-1,OE.2-.2', 'Pruned WN', ['experiments/imagenet100/cascade/pruned-wn/softpred_oe_R0/exp.result',
                                                                 'experiments/imagenet100/cascade/pruned-wn/softpred_oe_R1/exp.result',
                                                                 'experiments/imagenet100/cascade/pruned-wn/softpred_oe_R2/exp.result',
                                                                 ]],
    ['Imagenet 100', 'CascadeDW', 'Soft1-1', 'Pruned WN', ['experiments/imagenet100/cascade/pruned-wn/softpred_R0_depthweight/exp.result',
                                                          'experiments/imagenet100/cascade/pruned-wn/softpred_R1_depthweight/exp.result',
                                                          'experiments/imagenet100/cascade/pruned-wn/softpred_R3_depthweight/exp.result', ]],
    ['Imagenet 100', 'CascadeDW', 'Soft1-1,OE.2-.2', 'Pruned WN', ['experiments/imagenet100/cascade/pruned-wn/softpred_oe_R0_depthweight/exp.result',
                                                                  'experiments/imagenet100/cascade/pruned-wn/softpred_oe_R1_depthweight/exp.result',
                                                                  'experiments/imagenet100/cascade/pruned-wn/softpred_oe_R2_depthweight/exp.result',
                                                                 ]],
    ['Imagenet 100', 'CascadeDW', 'Soft1-1', '2 Lvl WN', ['experiments/imagenet100/cascade/two-lvl-wn/softpred_R0_depthweight/exp.result',
                                                         'experiments/imagenet100/cascade/two-lvl-wn/softpred_R1_depthweight/exp.result',
                                                         'experiments/imagenet100/cascade/two-lvl-wn/softpred_R2_depthweight/exp.result',]],
    ['Imagenet 100', 'CascadeDW', 'Soft1-1', 'NBDT', ['experiments/imagenet100/cascade/nbdt-binary/softpred_R0_depthweight/exp.result',
                                                     'experiments/imagenet100/cascade/nbdt-binary/softpred_R1_depthweight/exp.result',
                                                     'experiments/imagenet100/cascade/nbdt-binary/softpred_R2_depthweight/exp.result',]],
    ['Imagenet 100', 'MOS', 'MOS', '2 Lvl WN', ['experiments/imagenet100/mos/LRp1_R0/exp.result',
                                               'experiments/imagenet100/mos/LRp1_R1/exp.result',
                                               'experiments/imagenet100/mos/LRp1_R2/exp.result',
                                               ]],
    ['Imagenet 100', 'MOS', 'MOS', 'MOS Groups', ['experiments/imagenet100/mos/imagenet1000-mos_R0/exp.result',
                                                 'experiments/imagenet100/mos/imagenet1000-mos_R1/exp.result',
                                                 'experiments/imagenet100/mos/imagenet1000-mos_R2/exp.result',
                                                 ]],
    # Balanced 100
    ['Balanced 100', 'Softmax', 'CE', None, ['experiments/balanced100/softmax/R0/exp.result',
                                             'experiments/balanced100/softmax/R1/exp.result',
                                             'experiments/balanced100/softmax/R2/exp.result',
                                            ]],
    ['Balanced 100', 'MOS', 'MOS', 'MOS Groups', ['experiments/balanced100/mos/R0/exp.result',
                                                  'experiments/balanced100/mos/R1/exp.result',
                                                  'experiments/balanced100/mos/R2/exp.result',
                                                 ]],
    ['Balanced 100', 'Cascade', 'Soft1-1', 'Pruned WN', ['experiments/balanced100/cascade/pruned-wn/R0_dw/exp.result',
                                                         'experiments/balanced100/cascade/pruned-wn/R1_dw/exp.result',
                                                         'experiments/balanced100/cascade/pruned-wn/R2_dw/exp.result',
                                                        ]],
    ['Balanced 100', 'Cascade', 'Soft1-1,OE.2-.2', 'Pruned WN', [
        'experiments/balanced100/cascade/pruned-wn/R0_oe/exp.result',
        'experiments/balanced100/cascade/pruned-wn/R1_oe/exp.result',
        'experiments/balanced100/cascade/pruned-wn/R2_oe/exp.result',
    ]],
]
model_idx = 1
res_idx = 4
loss_info = []
for einfo in exp_info:
    if 'softmax' in einfo[1].lower():
        loss_info.append([r"$\mathcal{L}_{CE}$", None])
    elif 'mos' in einfo[1].lower():
        loss_info.append([r"$\mathcal{L}_{MOS}$", None])
    else:
        lsplit = einfo[2].split(',')
        linfo = r""
        lranges = {'alpha': None,
                   'beta': None,
                   'gamma': None}
        for ns, s in enumerate(lsplit):
            rng = re.findall('\d+', s)
            if 'soft' in s.lower():
                linfo += r"$\mathcal{L}_\text{soft}$"
                lranges['alpha'] = f'{rng[0]}--{rng[1]}'
            elif 'oe' in s.lower():
                linfo += r"$\mathcal{L}_\text{out}$"
                lranges['gamma'] = f'{rng[0]}--{rng[1]}'
            elif 'syn' in s.lower():
                linfo += r"$\mathcal{L}_\text{syn}$"
                lranges['beta'] = f'{rng[0]}--{rng[1]}'
            else:
                print(s)
                raise ValueError("Unknown loss")
            if ns < len(lsplit)-1:
                linfo += ", "
        loss_info.append([linfo, lranges])

In [3]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.precision', 4)

In [4]:
exp_stats = []
ood_datasets = ['OOD','iNaturalist', 'SUN', 'Places', 'Textures', 'coarseid-coarseood', 'coarseid-fineood',
                'balanced100-coarseood', 'balanced100-mediumood', 'balanced100-fineood', 'balanced100-finemediumood',
               ]
ood_metrics = ['AUROC', 'TNR', 'AUOUT']
ood_labels = ['AUROC', 'FPR', 'AUPR']
for exp in exp_info:
    pred_acc = np.empty((len(exp[res_idx]))) * np.nan
    msp = np.empty((len(exp[res_idx]), len(ood_datasets), 3)) * np.nan
    odin = np.empty((len(exp[res_idx]), len(ood_datasets), 3)) * np.nan
    mos = np.empty((len(exp[res_idx]), len(ood_datasets), 3)) * np.nan
    pred_msp = np.empty((len(exp[res_idx]), len(ood_datasets), 3)) * np.nan
    pred_odin = np.empty((len(exp[res_idx]), len(ood_datasets), 3)) * np.nan
    entmean_msp = np.empty((len(exp[res_idx]), len(ood_datasets), 3)) * np.nan
    entmean_odin = np.empty((len(exp[res_idx]), len(ood_datasets), 3)) * np.nan
    entmin_msp = np.empty((len(exp[res_idx]), len(ood_datasets), 3)) * np.nan
    entmin_odin = np.empty((len(exp[res_idx]), len(ood_datasets), 3)) * np.nan
    entmax_msp = np.empty((len(exp[res_idx]), len(ood_datasets), 3)) * np.nan
    entmax_odin = np.empty((len(exp[res_idx]), len(ood_datasets), 3)) * np.nan
    res_list = [msp, odin,
                mos,
                pred_msp, pred_odin,
                entmean_msp, entmean_odin,
                entmin_msp, entmin_odin,
                entmax_msp, entmax_odin,
               ]
    res_desc = ['MSP', 'ODIN',
                'MOS',
                'Pred', 'Pred ODIN',
                'MeanEnt', 'Mean Entropy Temp Scaled',
                'MinEnt', 'Min Entropy Temp Scaled',
                'MaxEnt', 'Max Entropy Temp Scaled',
               ]
    for i, it in enumerate(exp[res_idx]):
        it_res = torch.load(it)
        # Check if metrics include other ood sets
        try:
            if isinstance(it_res['ood'], tuple):
#                 print('tuple')
                it_res['ood'] = it_res['ood'][0]
                if 'MOS' in it_res['ood'].keys():
#                     print('MOS')
                    if 'OOD' not in it_res['ood']['MOS'].keys(): 
                        it_res['ood']['MOS'] = {'OOD': it_res['ood']['MOS']}
                elif 'OOD' not in it_res['ood']['MSP'].keys():
                    it_res['ood']['MSP'] = {'OOD': it_res['ood']['MSP']}
                    it_res['ood']['ODIN'] = {'OOD': it_res['ood']['ODIN']}
            elif 'MOS' in it_res['ood'].keys():
                if 'OOD' not in it_res['ood']['MOS'].keys(): 
                    it_res['ood']['MOS'] = {'OOD': it_res['ood']['MOS']}
            elif 'OOD' not in it_res['ood']['MSP'].keys():
                it_res['ood']['MSP'] = {'OOD': it_res['ood']['MSP']}
                it_res['ood']['ODIN'] = {'OOD': it_res['ood']['ODIN']}
        except:
            import pdb; pdb.set_trace()
        if it_res['pred'] is None:
            pred_acc[i] = float(it_res['top1']) 
        elif isinstance(it_res['pred'],np.ndarray):
            if it_res['pred'].size > 1:
                pred_acc[i] = float(it_res['pred'][0])
            else:
                pred_acc[i] = float(it_res['pred'])
        elif len(it_res['pred'].size()) > 0:
            pred_acc[i] = float(it_res['pred'][0])
        else:
            pred_acc[i] = float(it_res['pred'])
        for didx, dname in enumerate(ood_datasets):
            if 'MOS' in it_res['ood'].keys():
                if dname not in it_res['ood']['MOS'].keys():
                    continue
            else:
                if dname not in it_res['ood']['MSP'].keys():
                    continue
            for ood_res, ood_method in zip(res_list, res_desc):
                for mi, met in enumerate(ood_metrics):
                    if 'softmax' in exp[model_idx].lower():
                        if ood_method.upper() in ['MSP', 'ODIN']:
                            if met == 'TNR':
                                ood_res[i][didx][mi] = 1.- float(it_res['ood'][ood_method][dname][met][0])
                            else:
                                ood_res[i][didx][mi] = float(it_res['ood'][ood_method][dname][met][0])
                    elif 'mos' in exp[model_idx].lower():
                        if ood_method.upper() in ['MOS']:
                            if met == 'TNR':
                                ood_res[i][didx][mi] = 1.- float(it_res['ood'][ood_method][dname][met][0])
                            else:
                                ood_res[i][didx][mi] = float(it_res['ood'][ood_method][dname][met][0])
                    else:
                        if ood_method.upper() not in ['MSP', 'ODIN', 'MOS']:
                            meth = 'ODIN' if (('Temp' in ood_method) or ('ODIN' in ood_method)) else 'MSP'
                            if 'max' in ood_method.lower():
                                meth_idx = -4
                            elif 'pred' in ood_method.lower():
                                meth_idx = -3
                            elif 'mean' in ood_method.lower():
                                meth_idx = -2
                            else:
                                meth_idx = -1
                            if met == 'TNR':
                                ood_res[i][didx][mi] = 1.- float(it_res['ood'][meth][dname][met][meth_idx])
                            else:
                                ood_res[i][didx][mi] = float(it_res['ood'][meth][dname][met][meth_idx])
    exp_stats.append({desc: r for desc, r in zip(res_desc, res_list)})
    exp_stats[-1]['acc'] = pred_acc

In [5]:
def get_resultsdf_forOODDSET(dset='OOD'):
    dset_options = ['OOD','iNaturalist', 'SUN', 'Places', 'Textures',
                    'coarseid-coarseood', 'coarseid-fineood',
                    'imagenet1000-coarseood', 'imagenet1000-mediumood', 'imagenet1000-fineood',
                    'balanced100-coarseood', 'balanced100-mediumood', 'balanced100-fineood', 'balanced100-finemediumood',
                    # NOTE ORDER MATTERS HERE!
               ]
    if dset not in dset_options:
        raise ValueError("Unknown OOD DSET")
    else:
        dset_idx = dset_options.index(dset)
    miindex =  pd.MultiIndex.from_product(
        [[],[],[],[]], names=['Dataset', 'Model', 'Loss', 'Hierarchy'])
    acccolumns = pd.MultiIndex.from_product([['Accuracy'],[''], ['Mean', 'Std']], names=['acc/ood','metric','value'])
    oodcolumns = pd.MultiIndex.from_product(
        [['MSP', 'ODIN','MOS', 'Pred', 'MinEnt', 'MeanEnt'], ['AUROC', 'FPR', 'AUPR'], ['Mean', 'Std']], names=['acc/ood','metric','value'])
    df = pd.DataFrame(index=miindex, columns=oodcolumns)
    dfacc = pd.DataFrame(index=miindex, columns=acccolumns)
    df = dfacc.join(df)
    for info, stats in zip(exp_info, exp_stats):
        if ('imagenet' in dset): continue
        if ('coarseid' in dset) and ('imagenet 100' not in info[0].lower()): continue
        if ('balanced' in dset) and ('balanced' not in info[0].lower()): continue
        stats_dict = {}
        for ood_desc in res_desc:
            # Ignore temp scaled
            if ood_desc in ['Pred ODIN', 'Mean Entropy Temp Scaled', 'Min Entropy Temp Scaled', 'Max Entropy Temp Scaled']:
                continue
            for met_idx, met_desc in enumerate(ood_labels):
                if stats[ood_desc].shape[1] <= dset_idx: continue
                stats_dict[(ood_desc, met_desc, 'Mean')] = 100. * stats[ood_desc][:, dset_idx, met_idx].mean()
                stats_dict[(ood_desc, met_desc, 'Std')] = 100. * stats[ood_desc][:, dset_idx, met_idx].std()
        stats_dict[('Accuracy', '', 'Mean')] = 100. * stats['acc'].mean()
        stats_dict[('Accuracy', '', 'Std')] = 100. * stats['acc'].std()
        hstr = info[3] if info[3] is not None else 'N/A'
        for k, v in stats_dict.items():
            df.loc[(info[0], info[1], info[2], hstr), k] = v
    return df

In [6]:
df_ood = get_resultsdf_forOODDSET('OOD')
display(df_ood)

acc/ood                                           Accuracy              MSP                                              ODIN                                                MOS                                               Pred                                             MinEnt                                            MeanEnt                                             MaxEnt                                            
metric                                                                AUROC             FPR             AUPR            AUROC              FPR             AUPR            AUROC              FPR             AUPR            AUROC              FPR             AUPR            AUROC              FPR             AUPR            AUROC              FPR             AUPR            AUROC              FPR              AUPR         
value                                                 Mean     Std     Mean    Std     Mean     Std     Mean     Std     Mean     Std     Mean     Std     Mean     Std     Mean     Std     Mean     Std     Mean     Std     Mean     Std     Mean     Std     Mean     Std     Mean     Std     Mean     Std     Mean     Std     Mean     Std     Mean     Std     Mean     Std     Mean     Std     Mean      Std     Mean      Std
Dataset      Model     Loss            Hierarchy                                                                                                                                                                                                                                                                                                                                                                                        
Imagenet 100 Softmax   CE              N/A         81.2610  0.5342  90.2465  0.623  44.7451  1.2756  67.1531  1.8115  93.1977  0.3628  33.0980  3.9946  71.7468  2.2204      NaN     NaN      NaN     NaN      NaN     NaN      NaN     NaN      NaN     NaN      NaN     NaN      NaN     NaN      NaN     NaN      NaN     NaN      NaN     NaN      NaN     NaN      NaN     NaN      NaN     NaN      NaN      NaN      NaN      NaN
             Cascade   Soft1-1         NBDT        78.9960  2.0390      NaN    NaN      NaN     NaN      NaN     NaN      NaN     NaN      NaN     NaN      NaN     NaN      NaN     NaN      NaN     NaN      NaN     NaN  89.6539  0.9706  50.3529  4.0901  63.9028  3.8669  91.7078  0.4675  35.8824  5.7547  71.2148  3.2933  91.6255  0.8477  40.3922  6.6262  69.2092  3.3909  85.1551  1.5356  74.3529   3.2617  46.2986   4.1603
                                       2 Lvl WN    78.3213  0.4249      NaN    NaN      NaN     NaN      NaN     NaN      NaN     NaN      NaN     NaN      NaN     NaN      NaN     NaN      NaN     NaN      NaN     NaN  89.1135  0.5310  52.1961  0.7337  61.9433  2.1340  90.5790  0.3003  49.2157  2.6902  64.2317  1.7605  91.1798  0.5143  44.1961  1.1379  66.1790  2.4193  87.7659  1.8744  80.5490  22.2819  48.0588  11.8426
                                       Pruned WN   78.3614  0.8684      NaN    NaN      NaN     NaN      NaN     NaN      NaN     NaN      NaN     NaN      NaN     NaN      NaN     NaN      NaN     NaN      NaN     NaN  89.0887  0.6541  53.7647  2.4358  60.7355  1.7610  92.6817  0.2411  27.8431  2.4572  76.3041  0.8298  92.3859  0.4303  37.9608  1.0361  73.3062  1.7227  86.5881  0.6249  67.2157   1.9244  50.6299   1.7946
                       Soft1-1,OE.2-.2 Pruned WN   83.0522  0.1202      NaN    NaN      NaN     NaN      NaN     NaN      NaN     NaN      NaN     NaN      NaN     NaN      NaN     NaN      NaN     NaN      NaN     NaN  92.1572  0.2577  34.7451  0.8716  72.5341  0.6980  94.2932  0.2721  16.6667  0.9670  82.7055  0.8074  93.9251  0.2712  24.1569  0.6396  80.8566  0.8378  91.4921  0.1297  38.8627   1.2971  68.7579   0.5620
             CascadeDW Soft1-1         Pruned WN   82.3775  0.0568      NaN    NaN      NaN     NaN      NaN     NaN      NaN     NaN      NaN     NaN      NaN     NaN      Na

In [7]:
df_iNaturalist = get_resultsdf_forOODDSET('iNaturalist')
df_SUN = get_resultsdf_forOODDSET('SUN')
df_Places = get_resultsdf_forOODDSET('Places')
df_Textures = get_resultsdf_forOODDSET('Textures')
df_coarseid_coarseood = get_resultsdf_forOODDSET('coarseid-coarseood')
df_coarseid_fineood = get_resultsdf_forOODDSET('coarseid-fineood')
df_balanced100_coarseood = get_resultsdf_forOODDSET('balanced100-coarseood')
df_balanced100_mediumood = get_resultsdf_forOODDSET('balanced100-mediumood')
df_balanced100_fineood = get_resultsdf_forOODDSET('balanced100-fineood')
df_balanced100_finemediumood = get_resultsdf_forOODDSET('balanced100-finemediumood')

In [8]:
print("coarseood")
display(df_coarseid_coarseood)
print("fineood")
display(df_coarseid_fineood)

coarseood


acc/ood                                           Accuracy              MSP                                               ODIN                                                MOS                                           Pred                                             MinEnt                                            MeanEnt                                             MaxEnt                                          
metric                                                                AUROC              FPR             AUPR            AUROC              FPR             AUPR            AUROC          FPR             AUPR            AUROC              FPR             AUPR            AUROC              FPR             AUPR            AUROC              FPR             AUPR            AUROC              FPR             AUPR        
value                                                 Mean     Std     Mean     Std     Mean     Std     Mean     Std     Mean     Std     Mean     Std     Mean     Std     Mean    Std  Mean     Std     Mean     Std     Mean     Std     Mean     Std     Mean     Std     Mean     Std     Mean     Std     Mean     Std     Mean     Std     Mean     Std     Mean     Std     Mean     Std     Mean     Std     Mean     Std
Dataset      Model     Loss            Hierarchy                                                                                                                                                                                                                                                                                                                                                                                   
Imagenet 100 Softmax   CE              N/A         81.2610  0.5342  92.6162  0.6653  38.9778  1.3525  69.3019  2.0693  95.8997  0.4733  26.1333  4.2847  74.2832  2.7425      NaN    NaN   NaN     NaN      NaN     NaN      NaN     NaN      NaN     NaN      NaN     NaN      NaN     NaN      NaN     NaN      NaN     NaN      NaN     NaN      NaN     NaN      NaN     NaN      NaN     NaN      NaN     NaN      NaN     NaN
             Cascade   Soft1-1         NBDT        78.9960  2.0390      NaN     NaN      NaN     NaN      NaN     NaN      NaN     NaN      NaN     NaN      NaN     NaN      NaN    NaN   NaN     NaN      NaN     NaN      NaN     NaN      NaN     NaN      NaN     NaN      NaN     NaN      NaN     NaN      NaN     NaN      NaN     NaN      NaN     NaN      NaN     NaN      NaN     NaN      NaN     NaN      NaN     NaN
                                       2 Lvl WN    78.3213  0.4249      NaN     NaN      NaN     NaN      NaN     NaN      NaN     NaN      NaN     NaN      NaN     NaN      NaN    NaN   NaN     NaN      NaN     NaN      NaN     NaN      NaN     NaN      NaN     NaN      NaN     NaN      NaN     NaN      NaN     NaN      NaN     NaN      NaN     NaN      NaN     NaN      NaN     NaN      NaN     NaN      NaN     NaN
                                       Pruned WN   78.3614  0.8684      NaN     NaN      NaN     NaN      NaN     NaN      NaN     NaN      NaN     NaN      NaN     NaN      NaN    NaN   NaN     NaN      NaN     NaN  91.0163  0.6701  49.2000  2.7086  61.7189  1.8799  96.7408  0.2507  18.7111  2.7049  81.0084  1.0008  94.9590  0.4597  30.5333  1.2269  76.5228  1.8135  88.2367  0.7019  64.4444  2.4205  50.4227  1.9240
                       Soft1-1,OE.2-.2 Pruned WN   83.0522  0.1202      NaN     NaN      NaN     NaN      NaN     NaN      NaN     NaN      NaN     NaN      NaN     NaN      NaN    NaN   NaN     NaN      NaN     NaN  94.9641  0.0386  27.2444  0.9758  75.7929  0.6110  98.1797  0.1254   6.4444  1.0461  87.9223  0.8410  97.1142  0.0687  14.7556  0.6379  85.4674  0.7764  94.2366  0.1262  31.3333  1.2555  71.6750  0.7836
             CascadeDW Soft1-1         Pruned WN   82.3775  0.0568      NaN     NaN      NaN     NaN      NaN     NaN      NaN     NaN      NaN     NaN      NaN     NaN      NaN    NaN   NaN     NaN      NaN     NaN  93.

fineood


acc/ood                                           Accuracy              MSP                                           ODIN                                              MOS                                              Pred                                             MinEnt                                             MeanEnt                                              MaxEnt                                          
metric                                                                AUROC           FPR            AUPR            AUROC             FPR            AUPR            AUROC              FPR            AUPR            AUROC              FPR             AUPR            AUROC              FPR              AUPR            AUROC              FPR              AUPR            AUROC              FPR             AUPR        
value                                                 Mean     Std     Mean     Std  Mean     Std    Mean     Std     Mean    Std     Mean     Std    Mean     Std     Mean     Std     Mean     Std    Mean     Std     Mean     Std     Mean     Std     Mean     Std     Mean     Std     Mean      Std     Mean     Std     Mean     Std     Mean      Std     Mean     Std     Mean     Std     Mean     Std     Mean     Std
Dataset      Model     Loss            Hierarchy                                                                                                                                                                                                                                                                                                                                                                                  
Imagenet 100 Softmax   CE              N/A         81.2610  0.5342  72.4741  0.3067  88.0  0.8165  4.9591  0.1653  72.9326  1.874  85.3333  2.0548  5.3707  0.5462      NaN     NaN      NaN     NaN     NaN     NaN      NaN     NaN      NaN     NaN      NaN     NaN      NaN     NaN      NaN      NaN      NaN     NaN      NaN     NaN      NaN      NaN      NaN     NaN      NaN     NaN      NaN     NaN      NaN     NaN
             Cascade   Soft1-1         NBDT        78.9960  2.0390      NaN     NaN   NaN     NaN     NaN     NaN      NaN    NaN      NaN     NaN     NaN     NaN      NaN     NaN      NaN     NaN     NaN     NaN      NaN     NaN      NaN     NaN      NaN     NaN      NaN     NaN      NaN      NaN      NaN     NaN      NaN     NaN      NaN      NaN      NaN     NaN      NaN     NaN      NaN     NaN      NaN     NaN
                                       2 Lvl WN    78.3213  0.4249      NaN     NaN   NaN     NaN     NaN     NaN      NaN    NaN      NaN     NaN     NaN     NaN      NaN     NaN      NaN     NaN     NaN     NaN      NaN     NaN      NaN     NaN      NaN     NaN      NaN     NaN      NaN      NaN      NaN     NaN      NaN     NaN      NaN      NaN      NaN     NaN      NaN     NaN      NaN     NaN      NaN     NaN
                                       Pruned WN   78.3614  0.8684      NaN     NaN   NaN     NaN     NaN     NaN      NaN    NaN      NaN     NaN     NaN     NaN      NaN     NaN      NaN     NaN     NaN     NaN  74.6313  0.9243  88.0000  4.3205   5.3551  0.2146  62.2385  0.7230  96.3333   1.6997   3.1188  0.0428  73.0879  0.9509  93.6667   1.6997   4.4602  0.3027  74.2239  1.1431  88.0000  3.5590   4.9173  0.5057
                       Soft1-1,OE.2-.2 Pruned WN   83.0522  0.1202      NaN     NaN   NaN     NaN     NaN     NaN      NaN    NaN      NaN     NaN     NaN     NaN      NaN     NaN      NaN     NaN     NaN     NaN  71.1058  1.9848  91.0000  1.6330   4.2391  0.2876  65.1449  1.4024  93.3333   0.4714   3.5046  0.1096  70.0075  1.9341  94.6667   0.9428   3.9339  0.2504  70.9080  1.8661  95.3333  1.6997   4.0258  0.2130
             CascadeDW Soft1-1         Pruned WN   82.3775  0.0568      NaN     NaN   NaN     NaN     NaN     NaN      NaN    NaN      NaN     NaN     NaN     NaN      NaN     NaN      NaN     NaN     NaN     NaN  76.7798  3.3787

In [9]:
miindex =  pd.MultiIndex.from_product(
    [[],[],[],[],[]], names=['Dataset', 'OOD DSet', 'Model', 'Loss', 'Hierarchy'])
acccolumns = pd.MultiIndex.from_product([['Accuracy'],[' '], ['Mean', 'Std']], names=['acc/ood','metric','value'])
oodmethcolumn = pd.MultiIndex.from_product([['Best AUROC'],['Method'], [' ']], names=['acc/ood','metric','value'])
oodcolumns = pd.MultiIndex.from_product(
    [['Best AUROC'], ['AUROC', 'FPR', 'AUPR'], ['Mean', 'Std']], names=['acc/ood','metric','value'])
df = pd.DataFrame(index=miindex, columns=acccolumns)
dfood = pd.DataFrame(index=miindex, columns=oodcolumns)
dfoodmeth = pd.DataFrame(index=miindex, columns=oodmethcolumn)
df = df.join(dfoodmeth)
df = df.join(dfood)
ood_dsets = ['OOD', 'iNaturalist', 'SUN', 'Places', 'Textures']
for info, stats in zip(exp_info, exp_stats):
    for ods_idx, ood_dset in enumerate(ood_dsets):
        stats_dict = {}
        best_auroc_method = ''
        best_auroc = -1.
        for ood_desc in res_desc:
                
            if stats[ood_desc][:, ods_idx, 0].mean() > best_auroc:
#                 if 'softmax' in info[1].lower():
#                     if ood_desc != 'MSP': continue
                best_auroc = stats[ood_desc][:, ods_idx, 0].mean()
                stats_dict[('Best AUROC', 'Method', ' ')] = ood_desc
                for met_idx, met_desc in enumerate(ood_labels):
                    stats_dict[('Best AUROC', met_desc, 'Mean')] = stats[ood_desc][:, ods_idx, met_idx].mean()
                    stats_dict[('Best AUROC', met_desc, 'Std')] = stats[ood_desc][:, ods_idx, met_idx].std()
        hstr = info[3] if info[3] is not None else 'N/A'
        for k, v in stats_dict.items():
            df.loc[(info[0], ood_dset, info[1], info[2], hstr), k] = v
    df.loc[(info[0], 'OOD', info[1], info[2], hstr), ('Accuracy', ' ', 'Mean')] = 100. * stats['acc'].mean()
    df.loc[(info[0], 'OOD', info[1], info[2], hstr), ('Accuracy', ' ', 'Std')] = 100. * stats['acc'].std()
df = df.sort_index()
display(df)

acc/ood                                                       Accuracy                        Best AUROC                                                
metric                                                                                            Method   AUROC             FPR            AUPR        
value                                                             Mean     Std                              Mean     Std    Mean     Std    Mean     Std
Dataset      OOD DSet    Model     Loss            Hierarchy                                                                                            
Balanced 100 OOD         Cascade   Soft1-1         Pruned WN   78.3143  3.9667                   MeanEnt  0.8202  0.0138  0.5244  0.0106  0.7263  0.0182
                                   Soft1-1,OE.2-.2 Pruned WN   81.8286  0.1017                   MeanEnt  0.8484  0.0042  0.4856  0.0028  0.7489  0.0091
                         MOS       MOS             MOS Groups  80.3524  0.2091                       MOS  0.8680  0.0035  0.5887  0.0076  0.6997  0.0031
                         Softmax   CE              N/A         80.8476  0.2266                      ODIN  0.8682  0.0023  0.4996  0.0292  0.7414  0.0122
             Places      Cascade   Soft1-1         Pruned WN       NaN     NaN                   MeanEnt  0.9337  0.0064  0.3438  0.0238  0.9710  0.0032
                                   Soft1-1,OE.2-.2 Pruned WN       NaN     NaN                   MeanEnt  0.9468  0.0020  0.2562  0.0092  0.9756  0.0016
                         MOS       MOS             MOS Groups      NaN     NaN                       MOS  0.9465  0.0011  0.3615  0.0079  0.9681  0.0022
                         Softmax   CE              N/A             NaN     NaN                      ODIN  0.9635  0.0009  0.1913  0.0040  0.9841  0.0001
             SUN         Cascade   Soft1-1         Pruned WN       NaN     NaN                   MeanEnt  0.9491  0.0056  0.2886  0.0262  0.9766  0.0030
                                   Soft1-1,OE.2-.2 Pruned WN       NaN     NaN                   MeanEnt  0.9590  0.0023  0.2050  0.0107  0.9802  0.0016
                         MOS       MOS             MOS Groups      NaN     NaN                       MOS  0.9556  0.0012  0.3088  0.0104  0.9718  0.0023
                         Softmax   CE              N/A             NaN     NaN                      ODIN  0.9749  0.0010  0.1311  0.0080  0.9885  0.0002
             Textures    Cascade   Soft1-1         Pruned WN       NaN     NaN                    MinEnt  0.9206  0.0053  0.3177  0.0233  0.9492  0.0030
                                   Soft1-1,OE.2-.2 Pruned WN       NaN     NaN                    MinEnt  0.9281  0.0036  0.2733  0.0068  0.9532  0.0017
                         MOS       MOS             MOS Groups      NaN     NaN                       MOS  0.9281  0.0010  0.4375  0.0066  0.9308  0.0023
                         Softmax   CE              N/A             NaN     NaN                      ODIN  0.9562  0.0012  0.2274  0.0075  0.9694  0.0006
             iNaturalist Cascade   Soft1-1         Pruned WN       NaN     NaN                   MeanEnt  0.9283  0.0150  0.3709  0.0315  0.9681  0.0061
                                   Soft1-1,OE.2-.2 Pruned WN       NaN     NaN                   MeanEnt  0.9363  0.0053  0.3173  0.0162  0.9681  0.0027
                         MOS       MOS             MOS Groups      NaN     NaN                       MOS  0.9359  0.0018  0.4334  0.0131  0.9611  0.0014
                         Softmax   CE              N/A             NaN     NaN                      ODIN  0.9589  0.0021  0.2333  0.0032  0.9819  0.0009
Imagenet 100 OOD         Cascade   Soft1-1         2 Lvl WN    78.3213  0.4249                   MeanEnt  0.9118  0.0051  0.4420  0.0114  0.6618  0.0242
                                                   NBDT        78.9960  2.0390  Mean Entropy Temp Scaled  0.9236  0.0050  0.3318  0.0646  0.7178  0.0309
                      